In [ ]:
from trajectorylib.ml.model import CNN
from trajectorylib.ml.data_process import SlidingWindowDataProcessor
from trajectorylib.ml.trainer import SlidingWindowTrainer
from sklearn.metrics import accuracy_score

In [ ]:
import os
import random
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

## Loading Data

In [ ]:
data_folder = '/home/trijya/Dataset/DrivingData/'

In [ ]:
import os
import random

data_folder = '/home/trijya/Dataset/DrivingData/'

# Get a list of all CSV files in the data folder with full paths
csv_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]

# Select 80% of the files randomly
training_set = random.sample(csv_files, int(len(csv_files) * 0.8))

print("Training set:")
print(training_set)

print("Number of files in training set:", len(training_set))
print("Total number of CSV files:", len(csv_files))

In [ ]:
# Get the remaining files for test set
test_set = [f for f in csv_files if f not in training_set]
print("Test set:")
print(test_set)

## Preparing data for the model

In [ ]:
window_size = 6
dataprocessor = SlidingWindowDataProcessor(training_set, window_size=window_size)
dataprocessor.process_file_list()

In [ ]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Create an instance of the CNN model
model = CNN(window_size=window_size, dropout_rate=0.2, n_features=dataprocessor.all_X[0].shape[1])

## Training the model

In [ ]:
result_folder = "acc_cnn_training"

In [ ]:
trainer = SlidingWindowTrainer(result_folder=result_folder, 
                               model = model, 
                               device = device,
                               data_processor=dataprocessor)

In [ ]:
len(trainer.data_processor.all_X)

In [ ]:
trainer.data_processor.all_X[0].shape

In [ ]:
# Define Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#Define Learning Rate Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor=0.1, patience=10, min_lr=1e-6)

In [ ]:
trainer.train(criterion=criterion, optimizer=optimizer, scheduler=scheduler, num_epochs=10)

## Making inferences on unseen datasets

In [ ]:
testdata_processor = SlidingWindowDataProcessor(test_set, window_size=window_size)
testdata_processor.process_file_list()
trainer.predict(testdata_processor)

## Making inferences on unseen datasets by loading ONNX

In [ ]:
trainer.predict_onnx(testdata_processor)